<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">


# Notebook for generating configuration for online subscriptions in IBM Watson OpenScale 

This notebook shows how to generate the following artefacts:

1. Configuration JSON needed to configure an IBM Watson OpenScale online subscription. This JSON also contains information related to fairness configuration.
2. Drift Configuration Archive

In order to use this notebook you need to do the following:

1. Read the training data into a pandas dataframe called "data_df".  There is sample code below to show how this can be done if the training data is in IBM Cloud Object Storage. 
2. Edit the below cells and provide the training data and fairness configuration information. 
3. Run the notebook. It will generate a JSON and a download link for the JSON will be present at the very end of the notebook.
4. Download the JSON by clicking on the link and upload it in the IBM AI OpenScale GUI.

If you have multiple models (deployments), you will have to repeat the above steps for each model (deployment).

**Note:** Please restart the kernel after executing below cell

In [ ]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install pyspark --no-cache | tail -n 1
!pip install lime --no-cache | tail -n 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
VERSION = "5.3.5"

In [ ]:
WOS_CREDENTIALS = {
    "url": "https://cpd-namespace1.apps.xxxxx..com",
    "username": "xxxxx",
    "password": "xxxxx",
    "version": "4.5"
}

WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS["instance_id"] = "wml_local"

### For scoring the payload, used while generating the OpenScale Explainability and Drift archives.

In [ ]:
space_uid = 'd3fb73aa-e9cb-434f-b828-94e54b22a2a6'
deployment_uid = '10a9bc60-2b2f-4298-a0a3-e9301a66930d'

# Read training data into a pandas data frame

The first thing that you need to do is to read the training data into a pandas dataframe called "data_df".  Given below is sample code for doing this if the training data is in IBM Cloud Object Storage.  Please edit the below cell and make changes so that you can read your training data from the location where it is stored.  Please ensure that the training data is present in a data frame called "data_df".

*Note: Pandas' read\_csv method converts the columns to its data types. If you want the column type to not be interpreted, specify the dtype param to read_csv method in this cell. More on this method [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)*

*Note: By default NA values will be dropped while computing training data distribution and training the drift archive. Please ensure to handle the NA values during Pandas' read\_csv method*

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

In [ ]:
import pandas as pd
data_df=pd.read_csv ("german_credit_data_biased_training.csv")
data_df.head()

## Select the services for which configuration information needs to be generated

This notebook has support to generaton configuration information related to fairness , explainability and drift service. The below can be used by the user to control service specific configuration information.

Details of the service speicifc flags available:

- enable_fairness : Flag to allow generation of fairness specific data distribution needed for configuration
- enable_explainability : Flag to allow generation of explainability specific information
- enable_drift: Flag to allow generation of drift detection model needed by drift service


service_configuration_support = { <br>
&nbsp;&nbsp;&nbsp;&nbsp;"enable_fairness": True,   
&nbsp;&nbsp;&nbsp;&nbsp;"enable_explainability": True,    
&nbsp;&nbsp;&nbsp;&nbsp;"enable_drift": False  
    }  



In [ ]:
service_configuration_support = {
    "enable_fairness": True,
    "enable_explainability": True,
    "enable_drift": True
}

## Training Data and Fairness Configuration Information

Please provide information about the training data which is used to train the model.  In order to explain the configuration better, let us first consider an example of a Loan Processing Model which is trying to predict whether a person should get a loan or not. The training data for such a model will potentially contain the following columns: Credit_History, Monthly_salary, Applicant_Age, Loan_amount, Gender, Marital_status, Approval.  The "Approval" column contains the target field (label column or class label) and it can have the following values: "Loan Granted", "Loan Denied" or "Loan Partially Granted".  In this model we would like to ensure that the model is not biased against Gender=Female or Gender=Transgender.  We would also like to ensure that the model is not biased against the age group 15 to 30 years or age group 61 to 120 years. 

For the above model, the configuration information that we need to provide is:

- class_label:  This is the name of the column in the training data dataframe (data_df) which contains the target field (also known as label column or the class label).  For the Loan Processing Model it would be "Approval".
- feature_columns: This is a comma separated list of column names which contain the feature column names (in the training data dataframe data_df).  For the Loan Processing model this would be: ["Credit_History", "Monthly_salary", "Applicant_Age", "Loan_amount", "Gender", "Marital_status"]
- categorical_columns: The list of column names (in data_df) which contain categorical values.  This should also include those columns which originally contained categorical values and have now been converted to numeric values. E.g., in the Loan Processing Model, the Marital_status column originally could have values: Single, Married, Divorced, Separated, Widowed.  These could have been converted to numeric values as follows: Single -> 0, Married -> 1, Divorced -> 2, Separated -> 3 and Widowed -> 4.  Thus the training data will have numeric values.  Please identify such columns as categorical.  Thus the list of categorical columns for the Loan Processing Model will be Credit_History, Gender and Marital_status. 

For the Loan Processing Model, this information will be provided as follows:

training_data_info = { <br>
&nbsp;&nbsp;&nbsp;&nbsp;"class_label": "Approval",   
&nbsp;&nbsp;&nbsp;&nbsp;"feature_columns": ["Credit_History", "Monthly_salary", "Applicant_Age", "Loan_amount", "Gender", "Marital_status"],    
&nbsp;&nbsp;&nbsp;&nbsp;"categorical_columns": ["Credit_History","Gender","Marital_status"]   
    }  
    
  **Note:** Please note that categorical columns selected should be subset of feature columns. If there are no categorical columns among the feature columns selected , please set "categorical_columns as [] or None"

Please edit the next cell and provide the above information for your model.

In [ ]:
feature_columns=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
cat_features=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
class_label = "Risk"

In [ ]:
training_data_info = {
    "class_label": class_label,
    "feature_columns": feature_columns,
    "categorical_columns": cat_features
}

## Specify the Model Type

In the next cell, specify the type of your model.  If your model is a binary classification model, then set the type to "binary". If it is a multi-class classifier then set the type to "multiclass". If it is a regression model (e.g., Linear Regression), then set it to "regression".

In [ ]:
#Set model_type. Acceptable values are:["binary","multiclass","regression"]
model_type = "binary"

## Specify the Fairness Configuration

You need to provide the following information for the fairness configuration: 

- fairness_attributes:  These are the attributes on which you wish to monitor fairness. In the Loan Processing Model, we wanted to ensure that the model is not baised against people of specific age group and people belonging to a specific gender.  Hence "Applicant_Age" and "Gender" will be the fairness attributes for the Loan Processing Model.
- With Indirect Bias support, you can also monitor protected attributes for fairness. The protected attributes are those attributes which are present in the training data but are not used to train the model. For example, sensitive attributes like gender, race, age may be present in training data but are not used for training. To check if there exists indirect bias with respect to some protected attribute due to possible correlation with some feature column, it can be specified in fairness configuration.
- type: The data type of the fairness attribute (e.g., float or int or double)
- minority:  The minority group for which we want to ensure that the model is not biased.  For the Loan Processing Model we wanted to ensure that the model is not biased against people in the age group 15 to 30 years & 61 to 120 years as well as people with Gender = Female or Gender = Transgender.  Hence the minority group for the fairness attribute "Applicant_Age" will be [15,30] and [61,120] and the minority group for fairness attribute "Gender" will be: "Female", "Transgender".  
- majority: The majority group for which the model might be biased towards.  For the Loan Processing Model, the majority group for the fairness attribute "Applicant_Age" will be [31,60], i.e., all the ages except the minority group.  For the fairness attribute "Gender" the majority group will be: "Male".  
- threshold:  The fairness threshold beyond which the Model is considered to be biased.  For the Loan Processing Model, let us say that the Bank is willing to tolerate the fact that Female and Transgender applicants will get upto 20% lesser approved loans than Males.  However, if the percentage is more than 20% then the Loan Processing Model will be considered biased.  E.g., if the percentage of approved loans for Female or Transgender applicants is say 25% lesser than those approved for Male applicants then the Model is to be considered as acting in a biased manner.  Thus for this scenario, the Fairness threshold will be 80 (100-20) (this is represented as a value normalized to 1, i.e., 0.8).  

The fairness attributes for Loan Processing Model will be specified as:

fairness_attributes = [  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"feature": "Applicant_Age",   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"type" : "int",   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"majority": [ [31,60] ],   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"minority": [ [15, 30], [61,120] ],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"threshold" : 0.8  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;},  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"feature": "Gender",   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"type" : "string",   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"majority": ["Male"],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"minority": ["Female", "Transgender"],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"threshold" : 0.8  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}  
&nbsp;&nbsp;&nbsp;&nbsp;]  

Please edit the next cell and provide the fairness configuration for your model.

In [ ]:
fairness_attributes = [{
                            "type" : "string",
                            "feature": "Sex", 
                            "majority": ['female'],
                            "minority": ['male'],
                           "threshold": 0.98
                       }]

## Specify the Favorable and Unfavorable class values

The second part of fairness configuration is about the favourable and unfavourable class values.  Recall that in the case of Loan Processing Model, the target field (label column or class label) can have the following values: "Loan Granted", "Loan Denied" and "Loan Partially Granted".  Out of these values "Loan Granted" and "Loan Partially Granted" can be considered as being favorable and "Loan Denied" is unfavorable.  In other words in order to measure fairness, we need to know the target field values which can be considered as being favourable and those values which can be considered as unfavourable.  

For the Loan Prediction Model, the values can be specified as follows:

parameters = {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"favourable_class" :  [ "Loan Granted", "Loan Partially Granted" ],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"unfavourable_class": [ "Loan Denied" ]  
&nbsp;&nbsp;&nbsp;&nbsp;}  

In case of a regression models, the favourable and unfavourable classes will be ranges.  For example, for a model which predicts medicine dosage, the favorable outcome could be between 80 ml to 120 ml or between 5 ml to 20 ml whereas unfavorable outcome will be values between 21 ml to 79ml.  For such a model, the favorable and unfavorable values will be specified as follows:
     
parameters = {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"favourable_class" :  [ [5, 20], [80, 120] ],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"unfavourable_class": [ [21, 79] ]  
&nbsp;&nbsp;&nbsp;&nbsp;}  

Please edit the next cell to provide information about your model.

In [ ]:
# For classification models use the below.
parameters = {
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"]
    }

## Specify the number of records which should be processed for Fairness

The final piece of information that needs to be provided is the number of records (min_records) that should be used for computing the fairness. Fairness checks runs hourly.  If min_records is set to 5000, then every hour fairness checking will pick up the last 5000 records which were sent to the model for scoring and compute the fairness on those 5000 records.  Please note that fairness computation will not start till the time that 5000 records are sent to the model for scoring.

If we set the value of "min_records" to a small number, then fairness computation will get influenced by the scoring requests sent to the model in the recent past. In other words, the model might be flagged as being biased if it is acting in a biased manner on the last few records, but overall it might not be acting in a biased manner.  On the other hand, if the "min_records" is set to a very large number, then we will not be able to catch model bias quickly. Hence the value of min_records should be set such that it is neither too small or too large.

Please updated the next cell to specify a value for min_records.

In [ ]:
# min_records = <Minimum number of records to be considered for preforming scoring>
min_records = 95
# max_records = <Maximum number of records to be considered while computing fairness> [OPTIONAL]
max_records = None

## End of Input 

You need not edit anything beyond this point.  Run the notebook and go to the very last cell.  There will be a link to download the JSON file (called: "Download training data distribution JSON file").  Download the file and upload it using the IBM AI OpenScale GUI.

*Note: drop_na parameter of TrainingStats object should be set to 'False' if NA values are taken care while reading the training data in the above cells*

In [ ]:
from ibm_watson_openscale.utils.training_stats import TrainingStats

enable_explainability = service_configuration_support.get("enable_explainability")
enable_fairness = service_configuration_support.get("enable_fairness")

if enable_explainability or enable_fairness:
    fairness_inputs = None
    if enable_fairness:
        fairness_inputs = {
                "fairness_attributes": fairness_attributes,
                "min_records" : min_records,
                "favourable_class" :  parameters["favourable_class"],
                "unfavourable_class": parameters["unfavourable_class"]
            }
        if max_records is not None:
            fairness_inputs["max_records"] = max_records
    
    input_parameters = {
        "label_column": training_data_info["class_label"],
        "feature_columns": training_data_info["feature_columns"],
        "categorical_columns": training_data_info["categorical_columns"],
        "fairness_inputs": fairness_inputs,  
        "problem_type" : model_type  
    }

    training_stats = TrainingStats(data_df,input_parameters, explain=enable_explainability, fairness=enable_fairness, drop_na=True)
    config_json = training_stats.get_training_statistics()
    config_json["notebook_version"] = VERSION
#print(config_json)

In [ ]:
print(config_json)

## [Optional] Score Function

This is required if you are configuring :
- Drift (for classification models), and/or 
- Explainability (for headless subscriptions)

Please update the score function which will be used for generating drift detection model which will used for drift detection. Also, if you have a headless subscription, this will be used to generate explain perturbations archive which be used for explanations. 

The output of the score function should be a 2 arrays :
1. Array of probabilities
2. Array of model prediction


Please note:
- User is expected to make sure that the data type of the "class label" column selected and the prediction column are same . For eg : If class label is numeric , the prediction array should also be numeric
- Each entry of a probability array should have all the probabities of the unique class lable .
  For eg: If the model_type=multiclass and unique class labels are A, B, C, D . Each entry in the probability array should be a array of size 4 . Eg : [ [0.50,0.30,0.10,0.10] ,[0.40,0.20,0.30,0.10]...]
- **Please update the score function below with the help of templates documented [here](https://github.com/IBM/watson-openscale-samples/blob/main/training%20statistics/Score%20function%20templates%20for%20drift%20detection.md)**

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.set.default_space(space_uid)

In [ ]:
#Update score function
def score(training_data_frame):
      
    #The data type of the label column and prediction column should be same .
    #User needs to make sure that label column and prediction column array should have the same unique class labels
    prediction_column_name = "predictedLabel"
    probability_column_name = "probability"
        
    feature_columns = list(training_data_frame.columns)
    training_data_rows = training_data_frame[feature_columns].values.tolist()
    
    payload_scoring = {
      wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
           "fields": feature_columns,
           "values": [x for x in training_data_rows]
      }]
    }

    score = wml_client.deployments.score(deployment_uid, payload_scoring)
    score_predictions = score.get('predictions')[0]

    prob_col_index = list(score_predictions.get('fields')).index(probability_column_name)
    predict_col_index = list(score_predictions.get('fields')).index(prediction_column_name)

    if prob_col_index < 0 or predict_col_index < 0:
        raise Exception("Missing prediction/probability column in the scoring response")

    import numpy as np
    probability_array = np.array([value[prob_col_index] for value in score_predictions.get('values')])
    prediction_vector = np.array([value[predict_col_index] for value in score_predictions.get('values')])

    return probability_array, prediction_vector

## [Optional] Generate explain perturbations archive

This is required only for `headless` subscriptions, i.e., subscriptions which are configured without a REST endpoint for scoring needs. For explain to work in such scenarios, you must run following code snippet which generates and scores perturbations required by explainability service to function correctly.

Output of this is an explainability perturbations archive which must be uploaded to IBM Watson OpenScale during explain monitor configuration.

In [ ]:
# Generate perturbations archive for headless subscriptions
if enable_explainability:
    import pandas as pd
    from ibm_wos_utils.explainability.utils.perturbations import Perturbations
    from ibm_wos_utils.joblib.utils.notebook_utils import create_archive

    perturbations=Perturbations(training_stats=config_json.get("explainability_configuration"), problem_type=model_type)
    perturbs_df = perturbations.generate_perturbations()
    perturbs_df.to_csv("perturbations.csv",index=False)

    # use score function to score generated perturbations
    predict_probability = score(perturbs_df)
    scored_perturbations = pd.DataFrame(
        {
            'probability': predict_probability[0].tolist(),
            'predicted_label': predict_probability[1].tolist()
        }
    )

    # generate archive using scored perturbations and provide a link to download
    display(create_archive(scored_perturbations.to_csv(index=False), "perturbations.csv", "explainability"))

In [ ]:
import json

print("Finished generating training distribution data")

# Create a file download link
import base64
from IPython.display import HTML

def create_download_link( title = "Download training data distribution JSON file", filename = "training_distribution.json"):  
    if enable_explainability or enable_fairness:
        output_json = json.dumps(config_json, indent=2)
        b64 = base64.b64encode(output_json.encode())
        payload = b64.decode()
        html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        return HTML(html)
    else:
        print("No download link generated as fairness/explainability services are disabled.")

create_download_link()

## Drift configuration archive generation

Following code snippet is used to generate artefacts required for configuring drift identification for a model in IBM Watson OpenScale. Output is a drift archive which contains:

- Drift Detection Model (used for Accuracy Drift detection for classification models)
- Data Constraints (used for Data Consistency Drift detection for classification/regression models)

In [ ]:
#Generate drift detection model
from ibm_wos_utils.drift.drift_trainer import DriftTrainer
enable_drift = service_configuration_support.get("enable_drift")
if enable_drift:
    drift_detection_input = {
        "feature_columns":training_data_info.get("feature_columns"),
        "categorical_columns":training_data_info.get("categorical_columns"),
        "label_column": training_data_info.get("class_label"),
        "problem_type": model_type
    }
    
    drift_trainer = DriftTrainer(data_df,drift_detection_input)
    if model_type != "regression":
        #Note: batch_size can be customized by user as per the training data size
        drift_trainer.generate_drift_detection_model(score, batch_size=data_df.shape[0], check_for_ddm_quality=False)
    
    #Note:
    # - Two column constraints are not computed beyond two_column_learner_limit(default set to 200)
    # - Categorical columns with large (determined by categorical_unique_threshold; default > 0.8) number of unique values relative to total rows in the column are discarded. 
    #User can adjust the value depending on the requirement

    # user_overrides - Used to override drift constraint learning to selectively learn 
    # constraints on feature columns. Its a list of configuration, each specifying 
    # whether to learn distribution and/or range constraint on given set of columns.
    # First configuration of a given column would take preference.
    # 
    # "constraint_type" can have two possible values : single|double - signifying 
    # if this configuration is for single column or two column constraint learning.
    #
    # "learn_distribution_constraint" : True|False - signifying whether to learn 
    # distribution constraint for given config or not.
    #
    # "learn_range_constraint" : True|False - signifying whether to learn range 
    # constraint for given config or not. Only applicable to numerical feature columns.
    # 
    # "features" : [] - provides either a list of feature columns to be governed by 
    # given configuration for constraint learning.
    # Its a list of strings containing feature column names if "constraint_type" is "single".
    # Its a list of list of strings containing feature column names if "constraint_type" if 
    # "double". If only one column name is provided, all of the two column constraints 
    # involving this column will be dictated by given configuration during constraint learning.
    # This list is case-insensitive.
    #
    # In the example below, first config block says do not learn distribution and range single 
    # column constraints for features "MARITAL_STATUS", "PROFESSION", "IS_TENT" and "age".
    # Second config block says do not learn distribution and range two column constraints 
    # where "IS_TENT", "PROFESSION", and "AGE" are one of the two columns. Whereas, specifically, 
    # do not learn two column distribution and range constraint on combination of "MARITAL_STATUS" 
    # and "PURCHASE_AMOUNT".
    # "user_overrides"= [
    #     {
    #         "constraint_type": "single",
    #         "learn_distribution_constraint": False,
    #         "learn_range_constraint": False,
    #         "features": [
    #           "MARITAL_STATUS",
    #           "PROFESSION",
    #           "IS_TENT",
    #           "age"
    #         ]
    #     },
    #     {
    #         "constraint_type": "double",
    #         "learn_distribution_constraint": False,
    #         "learn_range_constraint": False,
    #         "features": [
    #           [
    #             "IS_TENT"
    #           ],
    #           [
    #             "MARITAL_STATUS"
    #             "PURCHASE_AMOUNT"
    #           ],
    #           [
    #             "PROFESSION"
    #           ],
    #           [
    #             "AGE"
    #           ]
    #         ]
    #     }
    # ]
    
    drift_trainer.learn_constraints(
        two_column_learner_limit=200, categorical_unique_threshold=0.8, user_overrides=[])
    drift_trainer.create_archive()

In [ ]:
#Generate a download link for drift detection model
from IPython.display import HTML
import base64
import io

def create_download_link_for_ddm( title = "Download Drift detection model", filename = "drift_detection_model.tar.gz"):  
    
    #Retains stats information    
    if enable_drift:
        with open(filename,"rb") as file:
            ddm = file.read()
        b64 = base64.b64encode(ddm)
        payload = b64.decode()
        
        html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        return HTML(html)
    else:
        print("Drift Detection is not enabled. Please enable and rerun the notebook")

create_download_link_for_ddm()
